# AutoGrad : 自動微分
この章ではPyTorchの重要な機能であるAutoGrad（自動微分）について説明します． <br>AutoGradとはtorch.Tensorを利用した計算において，各計算で実行した演算を計算グラフとして保持しておくことで勾配計算を自動化したものです．<br><br>以下に記載したサンプルを実行しながら，理解していきましょう．

In [1]:
import torch
import numpy as np

PyTorchとNumPyをimportします．

## torch.Tensor
torch.TensorはPyTorchパッケージの中心的なクラスです．<br>
基本的にはnumpyのndarrayと似た足し算や掛け算，ドット積などが計算できる行列のようなものだと考えて大丈夫です．<br>
ただnumpyのndarrayと大きく違う点があり，`.requires_grad=True`としておくと，各演算が実行されるごとに勾配計算用の計算グラフを保持し，`.backward()`を実行すると自動で勾配を計算してくれます．

tensorを作成してそれに伴う計算を追跡し勾配を計算するために`requires_grad=True`を設定します．

In [2]:
# 値は全て1を代入された，2×2の行列を作成
x = torch.ones(2, 2, requires_grad=True)
print(x)

# リストからtorch.Tensorへの変換
x = torch.tensor([[1., 1], 
                  [1, 1]], requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


`torch.from_numpy()`によってnumpy.ndarrayからtorch.Tensorに変換することができます．<br>
defaultではrequires_gradはFalseに設定されているので，勾配を計算したいときは`requires_grad=True`に設定し直す必要があります．<br>
注意点としてint型では勾配計算ができないのでintで定義していてもfloatに変える必要があります．

In [3]:
# np.arrayで2×2の行列を作成(dtypeでfloat型を指定)
x = np.array([[1, 1],
                 [1, 1]], dtype=float)
print('numpy.ndarray : \n', x)

# numpy.ndarrayからtorch.Tensorに変換
x = torch.from_numpy(x)
print('torch.Tensor : \n', x)

# x.requires_gradはdefaultでFalse
print(x.requires_grad)

# 勾配計算を行いたい場合，requires_gradをTrueに設定する
x.requires_grad=True
print(x.requires_grad)

numpy.ndarray : 
 [[1. 1.]
 [1. 1.]]
torch.Tensor : 
 tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64)
False
True


実際にrequires_gradをTrueに設定したtensorでの演算を行います.<br>
grad_fnはyがどういった演算によって作成されたかを示しており，これによって計算を追跡，勾配の計算を可能にします．

In [4]:
y = x + 2
print(y)

y = x - 1
print(y)

y = x * 2
print(y)

y = x / 2
print(y)

tensor([[3., 3.],
        [3., 3.]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[0., 0.],
        [0., 0.]], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000]], dtype=torch.float64, grad_fn=<DivBackward0>)


## 勾配計算
次は勾配の計算方法を説明します．<br>

In [5]:
y = x + 2
z = y*y*3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], dtype=torch.float64, grad_fn=<MulBackward0>) tensor(27., dtype=torch.float64, grad_fn=<MeanBackward0>)


`.backward()`は基本的に単一のスカラーに対して利用可能です．<br>
ベクトルに利用する際は引数gradientに同じ要素数のベクトルを入力する必要があります．

In [6]:
# 勾配計算
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]], dtype=torch.float64)


手計算でこの結果を確認してみます．<br>
out Tensorを$o$とするとき，$o = \frac{1}{4} \sum_i z_i, z_i = 3(x_i + 2)^2$となっており，$z_i|_{x_i = 1}=27$となります．<br>
$o$を行列の各要素$x_i$で偏微分すると$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i + 2)$となり$\frac{\partial o}{\partial x_i} | _{x_i=1} = \frac{9}{2} = 4.5$となることから，自動微分が上手く行っていることがわかります． 

また`.backward(retain_graph=True)`を複数回続けて行うと勾配の値は加算されていきます．<br>
従って通常の学習においては勾配の値をリセットする処理を挟みます．

In [7]:
y = x + 2
z = y*y*3
out = z.mean()
out.backward(retain_graph=True)
print(x.grad)
out.backward(retain_graph=True)
print(x.grad)
out.backward(retain_graph=True)
print(x.grad)

tensor([[9., 9.],
        [9., 9.]], dtype=torch.float64)
tensor([[13.5000, 13.5000],
        [13.5000, 13.5000]], dtype=torch.float64)
tensor([[18., 18.],
        [18., 18.]], dtype=torch.float64)


`with torch.no_grad():`ブロック内では勾配計算が行われないため，ネットワークの評価時に利用されます.

In [8]:
print('x : ', x.requires_grad)
print('x**2 : ', (x**2).requires_grad)

with torch.no_grad():
  print('with torch.no_gradをつけたx**2 : ', (x**2).requires_grad)
  

x :  True
x**2 :  True
with torch.no_gradをつけたx**2 :  False


# 課題
前回のWhat is PyTorchで実装した平均二乗誤差関数を再利用します.<br>
NumPyのndarrayの形でxとyが入力として与えられた時，平均二乗誤差を計算し，その入力に対する勾配を出力する関数を実装してください．



In [15]:
def MSE(x, y):
  x = torch.tensor(x, device="cuda:0", requires_grad=True)
  y = torch.tensor(y, device="cuda:0", requires_grad=True)
  z = ((x - y)**2).sum(dim=1).mean()
  z.backward()  
  return x.grad, y.grad

In [10]:
from google.colab import drive # driveを接続
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
# drive中の課題ファイルのあるディレクトリに移動
%cd /content/gdrive/My Drive/先端人工知能論Ⅰ/FAI20210601/data
from test import *

/content/gdrive/My Drive/先端人工知能論Ⅰ/FAI20210601/data


In [16]:
test_MSE_grad(MSE)

OK
